In [ ]:
%matplotlib inline
import numpy
import numpy as np
from matplotlib import pyplot
from amuse.units import (units, constants)
from amuse.ext.orbital_elements import new_binary_from_orbital_elements
from amuse.ext.orbital_elements import orbital_elements_from_binary
from amuse.lab import Particles

from amuse.ext.protodisk import ProtoPlanetaryDisk
from amuse.lab import nbody_system
from amuse.lab import new_powerlaw_mass_distribution
from amuse.plot import scatter
from amuse.community.ph4.interface import ph4

def orbital_period(Mtot, a):
    return (((4 * numpy.pi**2) * a**3)/(constants.G * Mtot)).sqrt()

SMBH = Particles(1)
SMBH.name = "SMBH"
SMBH.mass = 4.154e+6 | units.MSun
SMBH.position = (0, 0, 0) | units.pc
SMBH.velocity = (0, 0, 0) | units.kms
SMBH.radius = 2*constants.G*SMBH.mass/constants.c**2

In [ ]:
def relative_orbital_velocity(distance, mass=SMBH.mass):
    return (constants.G*mass/distance).sqrt()

def plot(body):
    scatter(body.x.value_in(units.pc), body.y.value_in(units.pc), s=20, alpha=0.5)
    #scatter(SMBH.x.value_in(units.pc), SMBH.z.value_in(units.pc), s=80, c='r')
    pyplot.xlim(-r, r)
    pyplot.ylim(-r, r)
#     pyplot.colorbar()
#     pyplot.show()

def dist(body):
    return (body.x**2+body.y**2+body.z**2).sqrt()

In [ ]:
n_BHs = 100
alpha_IMF = -2.35
r=3

def make_BHdisk_around_SMBH(SMBH, n_BHs=n_BHs):
    R = r|units.pc
    Ndisk = n_BHs
    Rin = 0.1
    Rout = 1
    Pinner1 = orbital_period(SMBH.mass, Rin*R)
    converter1 = nbody_system.nbody_to_si(SMBH.mass.sum(), R)
    masses = new_powerlaw_mass_distribution(Ndisk, 1.0|units.MSun, 100.0|units.MSun, 2.35)
    BHdisk = ProtoPlanetaryDisk(Ndisk,
                              convert_nbody=converter1,
                              Rmin=Rin,
                              Rmax=Rout,
                              q_out=10.0,
                              discfraction=masses.sum()/SMBH.mass).result
    BHdisk.mass = masses
    BHdisk.name = "BH"
    BHdisk.radius = 2*constants.G*BHdisk.mass/constants.c**2
    BHdisk.remove_attribute_from_store('u')
    BHdisk.move_to_center()
    return BHdisk, Pinner1, converter1

def make_gasdisk_around_SMBH(SMBH):
    R = 10|units.pc
    Ndisk = 10000
    Rin = 0.1
    Rout = 1
    Pinner2 = orbital_period(SMBH.mass, Rin*R)
    Mdisk = 1e-3 * SMBH.mass
    converter2 = nbody_system.nbody_to_si(SMBH.mass.sum(), R)
    
    gasdisk = ProtoPlanetaryDisk(Ndisk,
                              convert_nbody=converter1,
                              Rmin=Rin,
                              Rmax=Rout,
                              q_out=10.0,
                              discfraction=Mdisk/SMBH.mass).result
    gasdisk.name = "gas disk "
    gasdisk.move_to_center()
    masses = Mdisk/float(Ndisk)
    gasdisk.mass = masses
    rho = 3.0 | (units.g/units.cm**3)
    gasdisk.radius = (gasdisk.mass/(4*rho))**(1./3.)
    return gasdisk, Pinner2, converter2

BHdisk, Pinner1, converter1 = make_BHdisk_around_SMBH(SMBH)
gasdisk, Pinner2, converter2 = make_gasdisk_around_SMBH(SMBH)

plot(BHdisk)
plot(gasdisk)
pyplot.legend([BHdisk[0].name, gasdisk[0].name], loc="best")

In [ ]:
gravity = ph4(converter1, number_of_workers=32)
Nbody = BHdisk.copy()
Nbody.add_particles(SMBH)
Nbody.dist = dist(Nbody)
gravity.particles.add_particles(Nbody)
channel = gravity.particles.new_channel_to(Nbody)

stopping_condition = gravity.stopping_conditions.collision_detection
stopping_condition.enable()
collision_radius_multiplication_factor = 1e5



def merge_two_stars(bodies, particles_in_encounter):
    com_pos = particles_in_encounter.center_of_mass()
    com_vel = particles_in_encounter.center_of_mass_velocity()
    d = (particles_in_encounter[0].position - particles_in_encounter[1].position)
    v = (particles_in_encounter[0].velocity - particles_in_encounter[1].velocity)
    print("Actually merger occurred:")
    print("Two BHs (M=",particles_in_encounter.mass.in_(units.MSun),
          ") collided with d=", d.length().in_(units.au))
    new_particle=Particles(1)
    new_particle.mass = particles_in_encounter.total_mass()
    new_particle.name = particles_in_encounter[np.argmax(particles_in_encounter.mass)].name
    new_particle.position = com_pos
    new_particle.velocity = com_vel
    new_particle.radius = particles_in_encounter.radius.sum()
    bodies.add_particles(new_particle)
    bodies.remove_particles(particles_in_encounter)
    
def resolve_collision(collision_detection, gravity, bodies):
    if collision_detection.is_set():
        print("Well, we have an actual collision between two or more BHs.")
        print("This happened at time=", gravity.model_time.in_(units.Myr))
        for ci in range(len(collision_detection.particles(0))): 
            encountering_particles = Particles(particles=[collision_detection.particles(0)[ci],
                                                          collision_detection.particles(1)[ci]])
            colliding_BHs = encountering_particles.get_intersecting_subset_in(bodies)
            merge_two_stars(bodies, colliding_BHs)
            bodies.synchronize_to(gravity.particles)

In [ ]:
def get_binaries(particles,hardness=10,G = constants.G):
    """
    returns the binaries in a particleset. binaries are selected according to a hardness criterion [hardness=10]
    This function returns the binaries as a list of i,j particles. Triple detection is not done.
    
    >>> from amuse import datamodel
    >>> m = [1,1,1] | units.MSun
    >>> x = [-1,1,0] | units.AU
    >>> y = [0,0,1000] | units.AU
    >>> z = [0,0,0] | units.AU
    >>> vx = [0,0,0] | units.kms
    >>> vy = [1.,-1.,0] | units.kms
    >>> vz = [0,0,0] | units.kms
    >>> particles = datamodel.create_particle_set( mass=m,x=x,y=y,z=z,vx=vx,vy=vy,vz=vz )
    >>> binaries = particles.get_binaries()
    >>> print len(binaries)
    1
    
    """
    n=len(particles)
    total_Ek=(0.5*particles.mass*(particles.vx**2+particles.vy**2+particles.vz**2)).sum()
    average_Ek=total_Ek/particles.mass.sum()
    max_mass=particles.mass.amax()
    limitE=hardness*average_Ek

    a=numpy.argsort(particles.dist.number)

    binaries=[]

    for i in range(1, n-1):
        j=i+1
        while j<n and (particles.x[a[j]]-particles.x[a[i]])<2*G*max_mass/limitE:
            r2=(particles.x[a[j]]-particles.x[a[i]])**2+ \
               (particles.y[a[j]]-particles.y[a[i]])**2+ \
               (particles.z[a[j]]-particles.z[a[i]])**2 
            v2=(particles.vx[a[j]]-particles.vx[a[i]])**2+ \
               (particles.vy[a[j]]-particles.vy[a[i]])**2+ \
               (particles.vz[a[j]]-particles.vz[a[i]])**2 
            r=r2**0.5
            eb=G*(particles.mass[a[i]]+particles.mass[a[j]])/r-0.5*v2
            if eb > limitE:
                binary=particles[[a[i],a[j]]].copy()
                binary.hardness=eb/average_Ek
                binaries.append(binary)
            j+=1  

    return binaries


In [ ]:
end_time = 1000.0 | units.Myr
model_time = 0 | units.Myr
time = [] | units.Myr
t_diag = 10| units.Myr
plot(BHdisk)
plot(SMBH)
pyplot.show()
while(model_time<end_time):
    dt = 0.1 | units.Myr
    model_time += dt
    Nbody.collision_radius = Nbody.radius * collision_radius_multiplication_factor
    gravity.evolve_model(model_time)
    resolve_collision(stopping_condition, gravity, Nbody)
    channel.copy()


    
    if model_time >= t_diag:
        t_diag += 10 | units.Myr

        print("Evolved to t=", gravity.model_time.in_(units.Myr),
              "N=", len(Nbody)-1,
              "mass=", (Nbody-SMBH).mass.sum().in_(units.MSun))
    time.append(model_time)
    
    b=get_binaries(Nbody)
    if len(b)>0:
        print(b[0])
        break
        
 
gravity.stop()

plot(Nbody-SMBH)
plot(SMBH)
pyplot.show()

In [ ]:
print(gravity.stopping_conditions.collision_detection.is_set())

In [ ]:
dist(Nbody-SMBH).in_(units.pc) < (SMBH.radius.in_(units.pc)*1e5)

In [ ]:
b1 = get_binaries(Nbody)
b2 = Nbody.get_binaries()
print(len(b1))
print(len(b2))